In [1]:
import requests
from bs4 import BeautifulSoup

# Step 1: Fetch the edit page to get the sectok
edit_url = "http://192.168.1.247/psyc_jobs/dokuwiki/doku.php?id=wiki:test&do=edit"
response = requests.get(edit_url)

# Step 2: Parse the sectok from the HTML
soup = BeautifulSoup(response.text, 'html.parser')
sectok = soup.find('input', {'name': 'sectok'})['value']

# Step 3: Prepare and send the POST request to edit the page
edit_data = {
    'sectok': sectok,
    'id': 'wiki:test2',
    'rev': '0',
    'date': '1724816599',
    'prefix': '.',
    'suffix': '',
    'changecheck': 'a119e534072584a0ea88cdea4664aecd',
    'target': 'section',
    'wikitext': 'New Content HereNew Content HereNew Content HereNew Content Here',  # Replace with your content
    'do[save]': '1'
}

response = requests.post(edit_url, data=edit_data)

# Step 4: Check if the edit was successful
if response.status_code == 200:
    print("Page edited successfully!")
else:
    print("Failed to edit the page.")


TypeError: 'NoneType' object is not subscriptable

In [ ]:
import requests
from bs4 import BeautifulSoup

def edit_page(career, content):
    """Fetch and update the page with the specified content."""
    edit_url = f"http://192.168.1.247/dokuwiki_basic/doku.php?id=wiki:{career}&do=edit"
    response = requests.get(edit_url)

    if response.status_code != 200:
        print(f"Failed to fetch edit page for '{career}'. Status code: {response.status_code}")
        return

    # Print each line in the content
    for line in content.split('\n'):
        if 'youtube' in line:
            print('Youtube Here!!')
        print(line)
        
    
    # input('Stopping here line 20')

    soup = BeautifulSoup(response.text, 'html.parser')
    sectok = soup.find('input', {'name': 'sectok'})
    if sectok:
        sectok = sectok['value']
    else:
        print(f"Failed to extract sectok for page: {career}")
        return

    edit_data = {
        'sectok': sectok,
        'id': f'wiki:{career}',
        'rev': '0',
        'date': '1724816599',
        'prefix': '.',
        'suffix': '',
        'changecheck': 'a119e534072584a0ea88cdea4664aecd',
        'target': 'section',
        'wikitext': content,
        'do[save]': '1'
    }

    response = requests.post(edit_url, data=edit_data)

    if response.status_code == 200:
        print(f"Page '{career}' edited successfully!")
    else:
        print(f"Failed to edit the page '{career}'. Status code: {response.status_code}")


# Example usage
career = 'Big_Career'
content = 'New Content HereNew Content HereNew Content HereNew Content Here'
edit_page(career, content)


New Content HereNew Content HereNew Content HereNew Content Here
Page 'Big_Career' edited successfully!


In [1]:
import re
import requests
from bs4 import BeautifulSoup
from docx import Document

def read_docx(file_path, start_line, end_line):
    doc = Document(file_path)
    lines = []
    line_count = 0
    for paragraph in doc.paragraphs:
        if start_line <= line_count < end_line:
            lines.append(paragraph.text.strip())
        line_count += 1
    return lines

def parse_lines(lines):
    url_pattern = re.compile(r'https?://\S+')
    data = []
    current_main_category = None
    current_job = None
    jobs = {}

    for line in lines:
        if url_pattern.search(line):
            # Extract the URL
            url_match = url_pattern.search(line)
            url = url_match.group()
            
            # Extract the category (everything before the URL)
            category = line[:url_match.start()].strip()

            if current_job and current_job in jobs:
                jobs[current_job]['links'].append({
                    'url': url,
                    'category': category
                })
        elif line.lower().startswith("undefined"):
            # Skip lines starting with "undefined"
            continue
        else:
            if current_main_category is None:
                # Set the main category
                current_main_category = line
            elif current_job is None:
                # Set the job title (sub_category)
                current_job = line
                jobs[current_job] = {
                    'links': []
                }
            else:
                # Handle a new main category if a new line appears
                if line.strip() == "":
                    if current_main_category:
                        # Save the current main category and its jobs
                        data.append({
                            'main_category': current_main_category,
                            'jobs': jobs
                        })
                        # Reset for the next main category
                        current_main_category = None
                        jobs = {}
                        current_job = None
                else:
                    # If it's neither a URL nor an empty line, it might be a new job
                    if current_job:
                        # Make sure to add the current job to jobs before changing
                        current_job = line
                        jobs[current_job] = {
                            'links': []
                        }

    # Handle the last main category and jobs if they exist
    if current_main_category and jobs:
        data.append({
            'main_category': current_main_category,
            'jobs': jobs
        })

    return data

def format_content(links):
    """Format the content for the page including links."""
    content = "Here are some useful links:\n\n"
    for link in links:
        content += f"{link['category']}: {link['url']}\n"
    return content.strip()

def edit_page(career, content):
    """Fetch and update the page with the specified content."""
    edit_url = f"http://192.168.1.247/psyc_jobs/dokuwiki/doku.php?id=wiki:{career}&do=edit"
    response = requests.get(edit_url)

    if response.status_code != 200:
        print(f"Failed to fetch edit page for '{career}'. Status code: {response.status_code}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    sectok = soup.find('input', {'name': 'sectok'})
    if sectok:
        sectok = sectok['value']
    else:
        print(f"Failed to extract sectok for page: {career}")
        return

    edit_data = {
        'sectok': sectok,
        'id': f'wiki:{career}',
        'rev': '0',
        'date': '1724816599',
        'prefix': '.',
        'suffix': '',
        'changecheck': 'a119e534072584a0ea88cdea4664aecd',
        'target': 'section',
        'wikitext': content,
        'do[save]': '1'
    }

    response = requests.post(edit_url, data=edit_data)

    if response.status_code == 200:
        print(f"Page '{career}' edited successfully!")
    else:
        print(f"Failed to edit the page '{career}'. Status code: {response.status_code}")

def create_pages_from_docx(file_path, start_line, end_line):
    lines = read_docx(file_path, start_line, end_line)
    parsed_data = parse_lines(lines)

    for category in parsed_data:
        for job, details in category['jobs'].items():
            content = format_content(details['links'])
            edit_page(job, content)

# Specify the file path and line range
file_path = 'careers.docx'
start_line = 171  # Starting line (inclusive)
end_line = start_line + 3000  # Adjust the ending line as needed

create_pages_from_docx(file_path, start_line, end_line)


Failed to fetch edit page for 'Advertising Sales Representative / Agent / Executive'. Status code: 404
Failed to fetch edit page for 'Assistant Bank Manager'. Status code: 404
Failed to fetch edit page for 'Claims Supervisor'. Status code: 404
Failed to fetch edit page for 'Customer Service Representative'. Status code: 404
Failed to fetch edit page for 'Customer Service Representative Supervisor'. Status code: 404
Failed to fetch edit page for 'Department Manager'. Status code: 404
Failed to fetch edit page for 'Financial Advisor'. Status code: 404
Failed to fetch edit page for 'Fund Raiser'. Status code: 404
Failed to fetch edit page for 'General Operations Manager'. Status code: 404
Failed to fetch edit page for 'Loan Counselor'. Status code: 404
Failed to fetch edit page for 'Loan Officer'. Status code: 404
Failed to fetch edit page for 'Loss Prevention Manager'. Status code: 404
Failed to fetch edit page for 'Management Analyst'. Status code: 404
Failed to fetch edit page for 'Mar

## Create docuwiki page
 {{videosite>videoid?parameter1&parameter2|title}}
 see: https://www.dokuwiki.org/plugin:vshare
 

In [1]:
import re
import requests
from bs4 import BeautifulSoup
from docx import Document

def read_docx(file_path, start_line, end_line):
    doc = Document(file_path)
    lines = []
    line_count = 0
    for paragraph in doc.paragraphs:
        if start_line <= line_count < end_line:
            lines.append(paragraph.text.strip())
        line_count += 1
    return lines

def parse_lines(lines):
    url_pattern = re.compile(r'https?://\S+')
    data = []
    current_main_category = None
    current_job = None
    jobs = {}

    for line in lines:
        if url_pattern.search(line):
            # Extract the URL
            url_match = url_pattern.search(line)
            url = url_match.group()
            
            # Extract the category (everything before the URL)
            category = line[:url_match.start()].strip()

            if current_job and current_job in jobs:
                jobs[current_job]['links'].append({
                    'url': url,
                    'category': category
                })
        elif line.lower().startswith("undefined"):
            # Skip lines starting with "undefined"
            continue
        else:
            if current_main_category is None:
                # Set the main category
                current_main_category = line
            elif current_job is None:
                # Set the job title (sub_category)
                current_job = line
                jobs[current_job] = {
                    'links': []
                }
            else:
                # Handle a new main category if a new line appears
                if line.strip() == "":
                    if current_main_category:
                        # Save the current main category and its jobs
                        data.append({
                            'main_category': current_main_category,
                            'jobs': jobs
                        })
                        # Reset for the next main category
                        current_main_category = None
                        jobs = {}
                        current_job = None
                else:
                    # If it's neither a URL nor an empty line, it might be a new job
                    if current_job:
                        # Make sure to add the current job to jobs before changing
                        current_job = line
                        jobs[current_job] = {
                            'links': []
                        }

    # Handle the last main category and jobs if they exist
    if current_main_category and jobs:
        data.append({
            'main_category': current_main_category,
            'jobs': jobs
        })

    return data
##TODO: Add double back slash at end of each line!!!!!
def format_content(links):
    """Format the content for the page including links."""
    content = "Here are some useful links:\n\n"
    for link in links:
        content += f"{link['category']}: {link['url']}\n"
        if 'youtube' in link['url']:
            # Extract the video ID from the YouTube URL
            try:
                vid_number = link['url'].split('v=')[1]
                vid_number = vid_number.split('&')[0]  # In case there's more parameters after the video ID
                # waithere=input('Waiting at line 87')

                # Embed the video in the required format
                content += f"{{{{youtube>{vid_number}}}}}\n"
                print('content',content)
                # waithere=input('Waiting at line 87')
            except:
                pass
    return content.strip()

def edit_page(career, content):
    """Fetch and update the page with the specified content."""
    edit_url = f"http://192.168.1.247/dokuwiki_basic/doku.php?id=wiki:{career}&do=edit"
    response = requests.get(edit_url)

    if response.status_code != 200:
        print(f"Failed to fetch edit page for '{career}'. Status code: {response.status_code}")
        return

    # Print each line in the content
    for line in content.split('\n'):
        if 'youtube' in line:
            print('Foudn youtube')
            print('content',content)
        print(line)
    
    # input('Stopping here line 101')

    soup = BeautifulSoup(response.text, 'html.parser')
    sectok = soup.find('input', {'name': 'sectok'})
    if sectok:
        sectok = sectok['value']
    else:
        print(f"Failed to extract sectok for page: {career}")
        return

    edit_data = {
        'sectok': sectok,
        'id': f'wiki:{career}',
        'rev': '0',
        'date': '1724816599',
        'prefix': '.',
        'suffix': '',
        'changecheck': 'a119e534072584a0ea88cdea4664aecd',
        'target': 'section',
        'wikitext': content,
        'do[save]': '1'
    }

    response = requests.post(edit_url, data=edit_data)

    if response.status_code == 200:
        print(f"Page '{career}' edited successfully!")
    else:
        print(f"Failed to edit the page '{career}'. Status code: {response.status_code}")


def create_pages_from_docx(file_path, start_line, end_line):
    lines = read_docx(file_path, start_line, end_line)
    parsed_data = parse_lines(lines)

    for category in parsed_data:
        for job, details in category['jobs'].items():
            content = format_content(details['links'])
            edit_page(job, content)

# Specify the file path and line range
file_path = 'careers.docx'
start_line = 171  # Starting line (inclusive)
end_line = start_line + 3000  # Adjust the ending line as needed

create_pages_from_docx(file_path, start_line, end_line)


content Here are some useful links:

O*NET:: http://online.onetcenter.org/link/summary/41-3011.00
OOH:: http://www.bls.gov/ooh/sales/advertising-sales-agents.htm
Wild Card:: http://www.ehow.com/about_6123748_job-description-advertising-sales-rep.html
Video:: https://www.drkit.org/account-executive/
Video:: http://www.youtube.com/watch?v=YxH7xG8WzvY
{{youtube>YxH7xG8WzvY}}

content Here are some useful links:

O*NET:: http://online.onetcenter.org/link/summary/41-3011.00
OOH:: http://www.bls.gov/ooh/sales/advertising-sales-agents.htm
Wild Card:: http://www.ehow.com/about_6123748_job-description-advertising-sales-rep.html
Video:: https://www.drkit.org/account-executive/
Video:: http://www.youtube.com/watch?v=YxH7xG8WzvY
{{youtube>YxH7xG8WzvY}}
Video:: https://www.youtube.com/watch?v=ovjmCc1R6EU
{{youtube>ovjmCc1R6EU}}

Here are some useful links:

O*NET:: http://online.onetcenter.org/link/summary/41-3011.00
OOH:: http://www.bls.gov/ooh/sales/advertising-sales-agents.htm
Wild Card:: http:/